In [113]:
import pandas as pd
import numpy as np

import statistics

import pingouin as pt
from scipy.stats import f_oneway

import time
import math

import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

# Data structuring

#### Education

In [250]:
data_raw_education = pd.read_csv("education_survey_data_090421.csv")
data_education = data_raw_education.copy()
data_education.shape

(217, 79)

In [251]:
# Drop unnecessary header rows
data_education = data_education[2:]
data_education = data_education.reset_index(drop=True)

In [252]:
cutoff_education = datetime.datetime.strptime('2021-08-23 22:50:00', "%Y-%m-%d %H:%M:%S")
data_education['taken_in_appropiate_time_window'] = data_education['RecordedDate'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S") > cutoff_education)
data_education = data_education[data_education['taken_in_appropiate_time_window'] == True]
data_education.shape

(187, 80)

#### Housing

In [253]:
data_raw_housing = pd.read_csv("housing_survey_data_090421.csv")
data_housing = data_raw_housing.copy()
data_housing.shape

(128, 79)

In [254]:
# Drop unnecessary header rows
data_housing = data_housing[2:]
data_housing = data_housing.reset_index(drop=True)

In [255]:
cutoff_housing = datetime.datetime.strptime('2021-08-25 00:03:00', "%Y-%m-%d %H:%M:%S")
data_housing['taken_in_appropiate_time_window'] = data_housing['RecordedDate'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S") > cutoff_housing)
data_housing = data_housing[data_housing['taken_in_appropiate_time_window'] == True]
data_housing.shape

(120, 80)

### Combine data

In [256]:
# Stack data
data = data_education.append(data_housing, ignore_index=True)
data = data.reset_index(drop=True)
data.shape

(307, 80)

In [257]:
column_mapper = {
    "Q3.1": "prolific_id",
    "Duration (in seconds)": "duration",
    
    "Q5.7": "dt_q1",
    "Q5.13": "dt_q2",
    "Q5.14": "understood_system_did_well_dt",
    "Q5.15": "system_confusing_complicated_dt",
    "Q5.16": "system_confusing_confusing_dt",
    "Q5.17": "understood_system_easy_to_understand_dt",
    "Q5.18": "system_confusing_hard_to_remember_dt",
    "Q5.19": "understood_system_predictable_dt",
    "Q5.20": "understood_system_how_it_works_dt",
    "Q5.21": "understood_system_inputs_and_outputs_dt",
    
    "Q6.7": "lr_q1",
    "Q6.13": "lr_q2",
    "Q6.14": "understood_system_did_well_lr",
    "Q6.15": "system_confusing_complicated_lr",
    "Q6.16": "system_confusing_confusing_lr",
    "Q6.17": "understood_system_easy_to_understand_lr",
    "Q6.18": "system_confusing_hard_to_remember_lr",
    "Q6.19": "understood_system_predictable_lr",
    "Q6.20": "understood_system_how_it_works_lr",
    "Q6.21": "understood_system_inputs_and_outputs_lr",
    

    "Q7.7": "bb_no_shap_q1",
    "Q7.13": "bb_no_shap_q2",    
    "Q7.14": "understood_system_did_well_bb_no_shap",
    "Q7.15": "system_confusing_complicated_bb_no_shap",
    "Q7.16": "system_confusing_confusing_bb_no_shap",
    "Q7.17": "understood_system_easy_to_understand_bb_no_shap",
    "Q7.18": "system_confusing_hard_to_remember_bb_no_shap",
    "Q7.19": "understood_system_predictable_bb_no_shap",
    "Q7.20": "understood_system_how_it_works_bb_no_shap",
    "Q7.21": "understood_system_inputs_and_outputs_bb_no_shap",    
    
    "Q8.12": "bb_shap_q1",
    "Q8.18": "bb_shap_q2",
    "Q8.19": "understood_system_did_well_bb_shap",
    "Q8.20": "system_confusing_complicated_bb_shap",
    "Q8.21": "system_confusing_confusing_bb_shap",
    "Q8.22": "understood_system_easy_to_understand_bb_shap",
    "Q8.23": "system_confusing_hard_to_remember_bb_shap",
    "Q8.24": "understood_system_predictable_bb_shap",
    "Q8.25": "understood_system_how_it_works_bb_shap",
    "Q8.26": "understood_system_inputs_and_outputs_bb_shap"
}

In [258]:
data = data.rename(columns=column_mapper)

In [259]:
# Filter to only those with a valid value for prolific_id
data = data[(data.prolific_id.notnull()) & (data.prolific_id != "test")]
data.Finished.value_counts()

True     284
False     18
Name: Finished, dtype: int64

In [260]:
# Filter to only those who finished the survey
data = data[data.Finished=="True"]
data.shape

(284, 80)

In [261]:
# Filter only those that took > 4 minutes and were recorded after 2021-08-23 22:50:00
data.duration = pd.to_numeric(data.duration)
data = data[data.duration > 240]
data.shape
#time.strftime('%H:%M:%S', time.gmtime(math.ceil(data.duration.mean())))

(259, 80)

In [262]:
data = data[data['prolific_id'] != '5b9811290f2cf000010f31fb']
data = data[data['prolific_id'] != '5c57022635a46a0001682961']
data.shape

(257, 80)

In [263]:
data.head(10)

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Strongly agree,Strongly agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,Strongly agree,Strongly agree,Strongly agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Fail,First year grade,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,Somewhat agree,Somewhat disagree,Strongly disagree,Strongly agree,Somewhat agree,Neither agree nor disagree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Somewhat agree,Strongly disagree,Strongly disagree,Strongly agree,Somewhat disagree,Somewhat agree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Slightly knowledgeable,No,No,Yes,No,No,610c24152939c927ad421fa9,e

In [264]:
data.reset_index(inplace=True,drop=True)

In [265]:
# Average time to complete the survey
data.duration = pd.to_numeric(data.duration)
time.strftime('%H:%M:%S', time.gmtime(math.ceil(data.duration.mean())))

'00:09:12'

In [266]:
data.head()

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Strongly agree,Strongly agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,Strongly agree,Strongly agree,Strongly agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Fail,First year grade,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,Somewhat agree,Somewhat disagree,Strongly disagree,Strongly agree,Somewhat agree,Neither agree nor disagree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Somewhat agree,Strongly disagree,Strongly disagree,Strongly agree,Somewhat disagree,Somewhat agree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Slightly knowledgeable,No,No,Yes,No,No,610c24152939c927ad421fa9,e

In [267]:
print(data.dt_q1.value_counts(dropna=False))
print(data.lr_q1.value_counts(dropna=False))
print(data.bb_no_shap_q1.value_counts(dropna=False))
print(data.bb_shap_q1.value_counts(dropna=False))

NaN                                 134
Pass                                 44
Fail                                 35
Will have a high sales price         22
Will NOT have a high sales price     17
I don't know                          5
Name: dt_q1, dtype: int64
NaN                                 127
Pass                                 37
Fail                                 35
Will have a high sales price         23
Will NOT have a high sales price     22
I don't know                         13
Name: lr_q1, dtype: int64
NaN                                 144
Pass                                 40
Fail                                 28
Will have a high sales price         20
Will NOT have a high sales price     16
I don't know                          9
Name: bb_no_shap_q1, dtype: int64
NaN                                 115
Pass                                 57
Fail                                 32
Will NOT have a high sales price     29
Will have a high sales price      

In [268]:
print(data.dt_q2.value_counts(dropna=False))
print(data.lr_q2.value_counts(dropna=False))
print(data.bb_no_shap_q2.value_counts(dropna=False))
print(data.bb_shap_q2.value_counts(dropna=False))

NaN                                132
First year grade                    65
Size of living area                 15
House grade                         14
Size of house above ground          12
Mother's education level             9
Quality of family relationships      7
Number of absences                   1
Number of failures                   1
Property view rating                 1
Name: dt_q2, dtype: int64
NaN                                                  125
First year grade                                      72
House grade                                           44
Number of absences                                     5
Size of living area                                    4
Size of house above ground                             3
Property view rating                                   2
Average size of living area for closest 15 houses      2
Name: lr_q2, dtype: int64
NaN                                                  143
First year grade                             

## Add correct answers

In [269]:
data.bb_profile_1 = pd.to_numeric(data.bb_profile_1)
data.bb_profile_2 = pd.to_numeric(data.bb_profile_2)
data.int_profile_1 = pd.to_numeric(data.int_profile_1)
data.int_profile_1 = pd.to_numeric(data.int_profile_1)

In [270]:
correct_answers_raw = pd.read_csv("survey_analysis_profile_correct_answers.csv")
correct_answers = correct_answers_raw.copy()
correct_answers

,dataset_name,model,profile_number,q1_correct_answer,q2_correct_answer
0,education,dt,1,Fail,First year grade
1,education,dt,2,Pass,First year grade
2,education,dt,3,Fail,First year grade
3,education,dt,4,Pass,First year grade
4,education,lr,1,Fail,First year grade
5,education,lr,2,Pass,First year grade
6,education,lr,3,Fail,First year grade
7,education,lr,4,Pass,First year grade
8,education,shap,1,Fail,First year grade
9,education,shap,2,Pass,First year grade


In [271]:
# Join interpretable
temp = correct_answers.rename(columns={"model": "int_model",
                                       "profile_number":"int_profile_1",
                                       "q1_correct_answer": "int_q1_correct_answer",
                                       "q2_correct_answer": "int_q2_correct_answer"})
data = pd.merge(data,temp,on=['dataset_name','int_model','int_profile_1'],how='left')

# Join bb
# Join interpretable
temp = correct_answers.rename(columns={"model": "bb_model",
                                       "profile_number":"bb_profile_1",
                                       "q1_correct_answer":"bb_q1_correct_answer",
                                       "q2_correct_answer":"bb_q2_correct_answer"})
data = pd.merge(data,temp,on=['dataset_name','bb_model','bb_profile_1'],how='left')
data.head()

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window,int_q1_correct_answer,int_q2_correct_answer,bb_q1_correct_answer,bb_q2_correct_answer
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Strongly agree,Strongly agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,Strongly agree,Strongly agree,Strongly agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True,Fail,First year grade,Pass,First year grade
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Fail,First year grade,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True,Pass,First year grade,Fail,First year grade
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,Somewhat agree,Somewhat disagree,Strongly disagree,Strongly agree,Somewhat agree,Neither agree nor disagree,Strongly agree,Strongly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Somewhat agree,Strongly disagree,Strongly disagree,Strongly ag

In [272]:
data.tail()

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window,int_q1_correct_answer,int_q2_correct_answer,bb_q1_correct_answer,bb_q2_correct_answer
252,2021-09-01 18:55:05,2021-09-01 19:00:47,IP Address,139.62.222.185,100,342,True,2021-09-01 19:00:50,R_1dadIx4yDjfnQZG,NaN,NaN,NaN,NaN,30.3007049560546875,-81.44210052490234375,anonymous,EN,1,F_pa3ztC8FBCrg2hr,signature.png,15003,image/png,6101ed39f939f9662014e6a7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Will NOT have a high sales price,Property view rating,Neither agree nor disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Somewhat agree,Will NOT have a high sales price,Average size of living area for closest 15 houses,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Neither agree nor disagree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Not knowledgeable at all,No,No,No,No,No,6101ed39f939f9662014e6a7,housing,2,1,3,4,bb,no_shap,lr,True,Will have a high sales price,House grade,Will NOT have a high sales price,House grade
253,2021-09-02 12:25:19,2021-09-02 12:31:34,IP Address,12.68.149.132,100,375,True,2021-09-02 12:31:35,R_2P0YzYANRHDiJTa,NaN,NaN,NaN,NaN,33.0756988525390625,-96.7826995849609375,anonymous,EN,1,F_28D5soXkhDdOims,signature.png,11204,image/png,61080eaae202b0ef135056a5,Will NOT have a high sales price,Size of living area,Somewhat agree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Will have a high sales price,House grade,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,3.5,Slightly knowledgeable,No,No,No,No,No,61080eaae202b0ef135056a5,housing,3,1,2,3,int,shap,dt,True,Will NOT have a high sales price,Size of living area,Will have a high sales price,House grade
254,2021-09-03 09:29:40,2021-09-03 09:36:54,IP Address,184.182.196.66,100,433,True,2021-09-03 09:36:55,R_3n8wNbF1erPqeti,NaN,NaN,NaN,NaN,38.8878936767578125,-77.44480133056640625,anonymous,EN,1,F_x3L6HCetSbe4ZcR,signature.png,12313,image/png,6110

### Profile questions

In [273]:
def q1_is_correct(row,model_type):
    if model_type == 'bb_model':
        col_name = 'bb_' + row[model_type] + '_q1'
        correct_answer_col_name = 'bb_q1_correct_answer'
    elif model_type == 'int_model':
        col_name = row[model_type] + '_q1'
        correct_answer_col_name = 'int_q1_correct_answer'
    a = row[col_name]
    b = row[correct_answer_col_name]
    return (a==b)
        
def q2_is_correct(row,model_type):
    if model_type == 'bb_model':
        col_name = 'bb_' + row[model_type] + '_q2'
        correct_answer_col_name = 'bb_q2_correct_answer'
    elif model_type == 'int_model':
        col_name = row[model_type] + '_q2'
        correct_answer_col_name = 'int_q2_correct_answer'
    a = row[col_name].split(",")
    a.sort()
    b = row[correct_answer_col_name].split(",")
    b.sort()
    return (a==b)


data['q1_is_correct_int'] = data.apply(q1_is_correct,model_type='int_model',axis=1)
data['q1_is_correct_bb'] = data.apply(q1_is_correct,model_type='bb_model',axis=1)
data['q2_is_correct_int'] = data.apply(q2_is_correct,model_type='int_model',axis=1)
data['q2_is_correct_bb'] = data.apply(q2_is_correct,model_type='bb_model',axis=1)

data.head()

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window,int_q1_correct_answer,int_q2_correct_answer,bb_q1_correct_answer,bb_q2_correct_answer,q1_is_correct_int,q1_is_correct_bb,q2_is_correct_int,q2_is_correct_bb
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,Strongly agree,Strongly agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,Strongly agree,Strongly agree,Strongly agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True,Fail,First year grade,Pass,First year grade,True,True,True,True
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Fail,First year grade,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True,Pass,First year grade,Fail,First year grade,True,True,False,True
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,Somewhat agree,Somewhat disagree,Strongly disagree,Strongly agree,Somewhat agree,Neither agree nor disagree,Strongly agree,Strongly agree,NaN,NaN,NaN,N

### Likert questions

#### Recode likert scale questions

In [274]:
# Recode likert scale questions
likert_mapper = {
    "Strongly agree": 5,
    "Somewhat agree": 4,
    "Neither agree nor disagree": 3,
    "Somewhat disagree": 2,
    "Strongly disagree": 1
}

likert_reverse_mapper = {
    "Strongly agree": 1,
    "Somewhat agree": 2,
    "Neither agree nor disagree": 3,
    "Somewhat disagree": 4,
    "Strongly disagree": 5
}

In [275]:
system_understanding_likert_mappers = {
"understood_system_did_well": likert_mapper,
"system_confusing_complicated": likert_mapper,
"system_confusing_confusing": likert_mapper,
"understood_system_easy_to_understand": likert_mapper,
"system_confusing_hard_to_remember": likert_mapper,
"understood_system_predictable": likert_mapper,
"understood_system_how_it_works": likert_mapper,
"understood_system_inputs_and_outputs": likert_mapper
}

for q in system_understanding_likert_mappers:
    for name in data.columns:
        if q in name:
            data[name] = data[name].map(system_understanding_likert_mappers[q])
            # Convert to numeric
            #data[name] = pd.to_numeric(data[name])

#system_understanding_bb_columns = [col for col in data.columns if ('system_understanding' in col and '_bb' in col)]
#system_understanding_int_columns = [col for col in data.columns if ('system_understanding' in col and ('_dt' in col or '_lr' in col))]

In [276]:
data.head(5)

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window,int_q1_correct_answer,int_q2_correct_answer,bb_q1_correct_answer,bb_q2_correct_answer,q1_is_correct_int,q1_is_correct_bb,q2_is_correct_int,q2_is_correct_bb
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,5.0,5.0,4.0,2.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,5.0,5.0,5.0,2.0,4.0,4.0,4.0,4.0,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True,Fail,First year grade,Pass,First year grade,True,True,True,True
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,2.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,Fail,First year grade,4.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True,Pass,First year grade,Fail,First year grade,True,True,False,True
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,4.0,2.0,1.0,5.0,4.0,3.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,4.0,1.0,1.0,5.0,2.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Slightly knowledgeable,No,No,Yes,No,No,610c24152939c927ad421fa9,education,3,2,4,3,int,no_shap,dt,True,Pass,First year grade,Fail,First year grade,True,True,True,True
3,2021-08-24 00:12:19,2021-08-24 00:21:13,IP Address,24.165.25.108,100,534,True,2021-08-24 00:21:14,R_PAMuZ9Y8gMdN4S5,NaN,NaN,NaN,NaN,21.2816925048828125,-157.8249969482421875,anonymous,EN,0.89999

#### Get likert scale questions mean and standard devation

In [277]:
#likert_question_columns = [col for col in data.columns if 'system_understanding' in col]
#likert_questions = data.filter(items=likert_question_columns)
likert_questions = data.filter(regex='understood_system|system_confusing')
likert_questions

,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,2.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,2.0,4.0,4.0,4.0,4.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,4.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,2.0,1.0,5.0,4.0,3.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,1.0,5.0,2.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,4.0,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,3.0,4.0,4.0,3.0,4.0,4.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,2.0,4.0,3.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0
5,3.0,2.0,2.0,4.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,1.0,5.0,1.0,4.0,5.0,5.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,2.0,5.0,5.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,1.0,4.0,1.0,4.0,5.0,3.0
7,5.0,4.0,3.0,3.0,5.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,4.0,2.0,3.0,4.0,3.0,4.0
8,4.0,5.0,5.0,1.0,4.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,2.0,4.0,3.0,3.0,3.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,5.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
likert_questions.stack().std()

1.159802633944114

In [279]:
likert_questions.stack().mean()

3.450389105058366

In [280]:
likert_questions = ['understood_system_did_well',
                   'understood_system_easy_to_understand',
                   'understood_system_predictable',
                   'understood_system_how_it_works',
                   'understood_system_inputs_and_outputs',
                   'system_confusing_complicated',
                   'system_confusing_confusing',
                   'system_confusing_hard_to_remember']
models = ['dt','lr','bb_shap','bb_no_shap']

for m in models:
    for q in likert_questions:
        if m in ('dt','lr'):
            print(data[(data['int_model']==m)][f'{q}_{m}'].value_counts(dropna=False))
        if m in ('bb_shap','bb_no_shap'):
            print(data[(data['bb_model']==m.split('bb_')[1])][f'{q}_{m}'].value_counts(dropna=False))

4.0    58
5.0    53
3.0     8
2.0     4
1.0     2
Name: understood_system_did_well_dt, dtype: int64
4.0    46
2.0    35
3.0    17
1.0    16
5.0    11
Name: understood_system_easy_to_understand_dt, dtype: int64
4.0    56
2.0    29
3.0    23
5.0    10
1.0     7
Name: understood_system_predictable_dt, dtype: int64
4.0    70
5.0    28
2.0    14
3.0    10
1.0     3
Name: understood_system_how_it_works_dt, dtype: int64
4.0    63
5.0    44
3.0    10
2.0     6
1.0     2
Name: understood_system_inputs_and_outputs_dt, dtype: int64
4.0    44
5.0    31
2.0    30
3.0    12
1.0     8
Name: system_confusing_complicated_dt, dtype: int64
2.0    48
4.0    33
1.0    20
3.0    17
5.0     7
Name: system_confusing_confusing_dt, dtype: int64
4.0    48
2.0    28
5.0    22
3.0    20
1.0     7
Name: system_confusing_hard_to_remember_dt, dtype: int64
4.0    61
5.0    27
2.0    23
3.0    15
1.0     6
Name: understood_system_did_well_lr, dtype: int64
2.0    51
3.0    32
4.0    30
1.0    14
5.0     5
Name: understo

#### Cronbach's alpha

In [281]:
temp.melt()['value'].dropna().reset_index(drop=True)

0                             education
1                             education
2                             education
3                             education
4                             education
5                             education
6                             education
7                             education
8                             education
9                             education
10                            education
11                            education
12                            education
13                            education
14                            education
15                            education
16                              housing
17                              housing
18                              housing
19                              housing
20                              housing
21                              housing
22                              housing
23                              housing
24                              housing


In [282]:
# Understood system constuct
understood_system_likert_questions = pd.DataFrame()

#understood_system_did_well_cols = [col for col in data.columns if ('understood_system_did_well' in col)]
understood_system_items = ['understood_system_did_well',
                           'understood_system_easy_to_understand',
                           'understood_system_predictable',
                           'understood_system_how_it_works',
                           'understood_system_inputs_and_outputs']
for i in understood_system_items:
    temp = data.filter(regex=i)
    understood_system_likert_questions[i] = temp.melt()['value'].dropna()
    
pt.cronbach_alpha(data=understood_system_likert_questions)

(0.8185827632479856, array([0.793, 0.842]))

In [283]:
# Understood system constuct
system_confusing_likert_questions = pd.DataFrame()

#understood_system_did_well_cols = [col for col in data.columns if ('understood_system_did_well' in col)]
system_confusing_items = ['system_confusing_complicated',
                           'system_confusing_confusing',
                           'system_confusing_hard_to_remember']

for i in system_confusing_items:
    temp = data.filter(regex=i)
    system_confusing_likert_questions[i] = temp.melt()['value'].dropna()
    
pt.cronbach_alpha(data=system_confusing_likert_questions)

(0.775564901117629, array([0.74 , 0.807]))

#### Calculate system understanding

In [284]:
def calculate_mean_understood_system(x, model_type):
    if model_type == 'int_model':
        columns = [col for col in data.columns if ('understood_system' in col and ('_' + x['int_model']) in col)]
    elif model_type == 'bb_model':
        columns = [col for col in data.columns if ('understood_system' in col and ('_bb_' + x['bb_model']) in col)]
    total = 0
    n = 0
    for col in columns:
        total += x[col]
        n += 1
        
    mean = total/n    
    return mean

def calculate_mean_system_confusing(x, model_type):
    if model_type == 'int_model':
        columns = [col for col in data.columns if ('system_confusing' in col and ('_' + x['int_model']) in col)]
    elif model_type == 'bb_model':
        columns = [col for col in data.columns if ('system_confusing' in col and ('_bb_' + x['bb_model']) in col)]
    total = 0
    n = 0
    for col in columns:
        total += x[col]
        n += 1
        
    mean = total/n    
    return mean

In [285]:
data['understood_system_bb'] = data.apply(calculate_mean_understood_system,model_type='bb_model',axis=1)
data['understood_system_int'] = data.apply(calculate_mean_understood_system,model_type='int_model',axis=1)

data['system_confusing_bb'] = data.apply(calculate_mean_system_confusing,model_type='bb_model',axis=1)
data['system_confusing_int'] = data.apply(calculate_mean_system_confusing,model_type='int_model',axis=1)
data.head()

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window,int_q1_correct_answer,int_q2_correct_answer,bb_q1_correct_answer,bb_q2_correct_answer,q1_is_correct_int,q1_is_correct_bb,q2_is_correct_int,q2_is_correct_bb,understood_system_bb,understood_system_int,system_confusing_bb,system_confusing_int
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,5.0,5.0,4.0,2.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,5.0,5.0,5.0,2.0,4.0,4.0,4.0,4.0,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True,Fail,First year grade,Pass,First year grade,True,True,True,True,3.8,3.8,4.666667,3.666667
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,2.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,Fail,First year grade,4.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True,Pass,First year grade,Fail,First year grade,True,True,False,True,4.0,2.4,2.666667,4.000000
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,4.0,2.0,1.0,5.0,4.0,3.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,4.0,1.0,1.0,5.0,2.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Slightly knowledgeable,No,No,Yes,No,No,610c24152939c927ad421fa9,education,3,2,4,3,int,no_shap,dt,True,Pass,First year grade,Fail,First year grade,True,True,True,True,4.6,4.4,1.333333,2.333333
3,2021-08-24 00:12:19,2021-08-24 0

# Analysis

## Dataset structure

In [325]:
temp_int = data.filter(items=['dataset_name',
                              'prolific_id',
                              'int_model',
                              'q1_is_correct_int',
                              'q2_is_correct_int',
                              'understood_system_int',
                              'system_confusing_int'])
temp_int_col_names = [col_name.replace("_int","").replace("int_","") for col_name in temp_int.columns]
temp_int = temp_int.rename(columns=dict(zip(temp_int.columns, temp_int_col_names)))

temp_bb = data.filter(items=['dataset_name',
                              'prolific_id',
                              'bb_model',
                              'q1_is_correct_bb',
                              'q2_is_correct_bb',
                              'understood_system_bb',
                              'system_confusing_bb'])
temp_bb_col_names = [col_name.replace("_bb","").replace("bb_","") for col_name in temp_bb.columns]
temp_bb = temp_bb.rename(columns=dict(zip(temp_bb.columns, temp_bb_col_names)))

random_effects_df = pd.concat([temp_int,temp_bb])
random_effects_df['model_type'] = random_effects_df['model'].apply(lambda x: "int" if x in ('dt','lr') else "bb")
random_effects_df['q1_is_correct'] = random_effects_df['q1_is_correct'].apply(lambda x: 1 if x == True else 0)
random_effects_df['q2_is_correct'] = random_effects_df['q2_is_correct'].apply(lambda x: 1 if x == True else 0)


random_effects_df_education = random_effects_df[random_effects_df['dataset_name'] == "education"]
random_effects_df_housing = random_effects_df[random_effects_df['dataset_name'] == "housing"]

print(random_effects_df.head())
print(random_effects_df.tail())

  dataset_name               prolific_id model  q1_is_correct  q2_is_correct  \
0    education  6102d3c05c31cb5c38158b34    lr              1              1   
1    education  610b88a14eed14f2b8b17b86    lr              1              0   
2    education  610c24152939c927ad421fa9    dt              1              1   
3    education  60fd5178af4a40ee405f57cf    dt              1              1   
4    education  60fc576e3a4df1ba622a76a1    lr              1              1   

   understood_system  system_confusing model_type  
0                3.8          3.666667        int  
1                2.4          4.000000        int  
2                4.4          2.333333        int  
3                4.2          4.000000        int  
4                2.6          4.000000        int  
    dataset_name               prolific_id    model  q1_is_correct  \
252      housing  6101ed39f939f9662014e6a7  no_shap              1   
253      housing  61080eaae202b0ef135056a5     shap              1 

In [345]:
random_effects_df.head(10)

,dataset_name,prolific_id,model,q1_is_correct,q2_is_correct,understood_system,system_confusing,model_type
0,education,6102d3c05c31cb5c38158b34,lr,1,1,3.8,3.666667,int
1,education,610b88a14eed14f2b8b17b86,lr,1,0,2.4,4.000000,int
2,education,610c24152939c927ad421fa9,dt,1,1,4.4,2.333333,int
3,education,60fd5178af4a40ee405f57cf,dt,1,1,4.2,4.000000,int
4,education,60fc576e3a4df1ba622a76a1,lr,1,1,2.6,4.000000,int
5,education,6108bae61196a3bd6fc3a24b,dt,1,1,3.8,2.000000,int
6,education,60fd0dd1b8b6c7057d178986,lr,1,1,3.8,5.000000,int
7,education,610dd9b10888c8a8801619d0,dt,1,1,3.8,4.000000,int
8,education,60fe250b3984cfdf32c05860,dt,1,1,2.8,4.666667,int
9,education,60fd4b01d7a25b34ca38a180,lr,1,1,3.8,4.000000,int


# Int vs BB

### Education - task 1 and task 2 for int vs bb

In [326]:
#[[black box correct / interpretable correct, black box correct / interpretable incorrect],
# [black box incorrect / interpretable correct, black box incorrect / interpretable incorrect]]
edu_data = data[data['dataset_name'] == "education"]
v1 = edu_data[(edu_data['q1_is_correct_int'] == True) & (edu_data['q1_is_correct_bb'] == True)].shape[0]
v2 = edu_data[(edu_data['q1_is_correct_int'] == False) & (edu_data['q1_is_correct_bb'] == False)].shape[0]
v3 = edu_data[(edu_data['q1_is_correct_int'] == True) & (edu_data['q1_is_correct_bb'] == False)].shape[0]
v4 = edu_data[(edu_data['q1_is_correct_int'] == False) & (edu_data['q1_is_correct_bb'] == True)].shape[0]

table = [[v1,v4],
         [v3,v2]]

from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(table, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))

statistic=13.000, p-value=0.296


In [327]:
#[[black box correct / interpretable correct, black box correct / interpretable incorrect],
# [black box incorrect / interpretable correct, black box incorrect / interpretable corect]]
edu_data = data[data['dataset_name'] == "education"]
v1 = edu_data[(edu_data['q2_is_correct_int'] == True) & (edu_data['q2_is_correct_bb'] == True)].shape[0]
v2 = edu_data[(edu_data['q2_is_correct_int'] == False) & (edu_data['q2_is_correct_bb'] == False)].shape[0]
v3 = edu_data[(edu_data['q2_is_correct_int'] == True) & (edu_data['q2_is_correct_bb'] == False)].shape[0]
v4 = edu_data[(edu_data['q2_is_correct_int'] == False) & (edu_data['q2_is_correct_bb'] == True)].shape[0]

table = [[v1,v4],
         [v3,v2]]

from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(table, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))

statistic=9.000, p-value=0.087


### Housing - task 1 and task 2 for int vs bb

In [328]:
#[[black box correct / interpretable correct, black box correct / interpretable incorrect],
# [black box incorrect / interpretable correct, black box incorrect / interpretable corect]]
house_data = data[data['dataset_name'] == "housing"]
v1 = house_data[(house_data['q1_is_correct_int'] == True) & (house_data['q1_is_correct_bb'] == True)].shape[0]
v2 = house_data[(house_data['q1_is_correct_int'] == False) & (house_data['q1_is_correct_bb'] == False)].shape[0]
v3 = house_data[(house_data['q1_is_correct_int'] == True) & (house_data['q1_is_correct_bb'] == False)].shape[0]
v4 = house_data[(house_data['q1_is_correct_int'] == False) & (house_data['q1_is_correct_bb'] == True)].shape[0]

table = [[v1,v4],
         [v3,v2]]

from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(table, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))

statistic=13.000, p-value=0.296


In [329]:
#[[black box correct / interpretable correct, black box correct / interpretable incorrect],
# [black box incorrect / interpretable correct, black box incorrect / interpretable corect]]
house_data = data[data['dataset_name'] == "housing"]
v1 = house_data[(house_data['q2_is_correct_int'] == True) & (house_data['q2_is_correct_bb'] == True)].shape[0]
v2 = house_data[(house_data['q2_is_correct_int'] == False) & (house_data['q2_is_correct_bb'] == False)].shape[0]
v3 = house_data[(house_data['q2_is_correct_int'] == True) & (house_data['q2_is_correct_bb'] == False)].shape[0]
v4 = house_data[(house_data['q2_is_correct_int'] == False) & (house_data['q2_is_correct_bb'] == True)].shape[0]

table = [[v1,v4],
         [v3,v2]]

from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(table, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))

statistic=14.000, p-value=0.108


### Education - understood system, system confusing int vs bb

In [330]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [331]:
md = smf.mixedlm("understood_system ~ model_type", random_effects_df_education, groups=random_effects_df_education['prolific_id'])
mdf = md.fit()
print(mdf.summary())

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: understood_system
No. Observations: 320     Method:             REML             
No. Groups:       160     Scale:              0.4026           
Min. group size:  2       Log-Likelihood:     -360.7742        
Max. group size:  2       Converged:          Yes              
Mean group size:  2.0                                          
---------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept             3.765    0.060 62.816 0.000  3.648  3.882
model_type[T.int]    -0.119    0.071 -1.674 0.094 -0.258  0.020
Group Var             0.172    0.093                           



In [332]:
md = smf.mixedlm("system_confusing ~ model_type", random_effects_df_education, groups=random_effects_df_education['prolific_id'])
mdf = md.fit()
print(mdf.summary())

            Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: system_confusing
No. Observations: 320     Method:             REML            
No. Groups:       160     Scale:              0.7376          
Min. group size:  2       Log-Likelihood:     -455.7216       
Max. group size:  2       Converged:          Yes             
Mean group size:  2.0                                         
--------------------------------------------------------------
                     Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            2.965    0.081 36.740 0.000  2.806  3.123
model_type[T.int]    0.275    0.096  2.864 0.004  0.087  0.463
Group Var            0.304    0.124                           



### Housing - understood system, system confusing int vs bb

In [333]:
md = smf.mixedlm("understood_system ~ model_type", random_effects_df_housing, groups=random_effects_df_housing['prolific_id'])
mdf = md.fit()
print(mdf.summary())

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: understood_system
No. Observations: 194     Method:             REML             
No. Groups:       97      Scale:              0.4233           
Min. group size:  2       Log-Likelihood:     -226.6914        
Max. group size:  2       Converged:          Yes              
Mean group size:  2.0                                          
---------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept             3.522    0.080 43.849 0.000  3.364  3.679
model_type[T.int]    -0.229    0.093 -2.450 0.014 -0.412 -0.046
Group Var             0.202    0.130                           



In [334]:
md = smf.mixedlm("system_confusing ~ model_type", random_effects_df_housing, groups=random_effects_df_housing['prolific_id'])
mdf = md.fit()
print(mdf.summary())

            Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: system_confusing
No. Observations: 194     Method:             REML            
No. Groups:       97      Scale:              0.6508          
Min. group size:  2       Log-Likelihood:     -261.5644       
Max. group size:  2       Converged:          Yes             
Mean group size:  2.0                                         
--------------------------------------------------------------
                     Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            3.186    0.095 33.402 0.000  2.999  3.372
model_type[T.int]    0.419    0.116  3.619 0.000  0.192  0.646
Group Var            0.232    0.141                           



# 4 models individually

In [335]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Education

In [336]:
md = smf.mixedlm("q1_is_correct ~ model", random_effects_df_education, groups=random_effects_df_education['prolific_id'])
mdf = md.fit()
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: q1_is_correct
No. Observations: 320     Method:             REML         
No. Groups:       160     Scale:              0.0995       
Min. group size:  2       Log-Likelihood:     -127.1693    
Max. group size:  2       Converged:          Yes          
Mean group size:  2.0                                      
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.869    0.039 22.470 0.000  0.793  0.944
model[T.lr]      -0.091    0.055 -1.647 0.099 -0.199  0.017
model[T.no_shap]  0.088    0.054  1.633 0.102 -0.018  0.194
model[T.shap]    -0.067    0.050 -1.326 0.185 -0.165  0.032
Group Var         0.027    0.039                           



In [337]:
md = smf.mixedlm("q2_is_correct ~ model", random_effects_df_education, groups=random_effects_df_education['prolific_id'])
mdf = md.fit()
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: q2_is_correct
No. Observations: 320     Method:             REML         
No. Groups:       160     Scale:              0.0835       
Min. group size:  2       Log-Likelihood:     -85.9291     
Max. group size:  2       Converged:          Yes          
Mean group size:  2.0                                      
-----------------------------------------------------------
                  Coef. Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.782    0.034 23.042 0.000  0.715  0.848
model[T.lr]       0.155    0.049  3.183 0.001  0.060  0.250
model[T.no_shap]  0.093    0.049  1.911 0.056 -0.002  0.189
model[T.shap]     0.170    0.045  3.763 0.000  0.082  0.259
Group Var         0.013    0.030                           



In [338]:
md = smf.mixedlm("understood_system ~ model", random_effects_df_education, groups=random_effects_df_education['prolific_id'])
mdf = md.fit()
print(mdf.summary())

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: understood_system
No. Observations: 320     Method:             REML             
No. Groups:       160     Scale:              0.3645           
Min. group size:  2       Log-Likelihood:     -351.9458        
Max. group size:  2       Converged:          Yes              
Mean group size:  2.0                                          
---------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept             3.813    0.079 48.358 0.000  3.658  3.967
model[T.lr]          -0.346    0.110 -3.140 0.002 -0.562 -0.130
model[T.no_shap]      0.191    0.106  1.799 0.072 -0.017  0.399
model[T.shap]        -0.229    0.099 -2.314 0.021 -0.422 -0.035
Group Var             0.180    0.095                           



In [339]:
md = smf.mixedlm("system_confusing ~ model", random_effects_df_education, groups=random_effects_df_education['prolific_id'])
mdf = md.fit()
print(mdf.summary())

            Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: system_confusing
No. Observations: 320     Method:             REML            
No. Groups:       160     Scale:              0.6840          
Min. group size:  2       Log-Likelihood:     -445.7812       
Max. group size:  2       Converged:          Yes             
Mean group size:  2.0                                         
--------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            3.050    0.106 28.698 0.000  2.842  3.258
model[T.lr]          0.394    0.150  2.629 0.009  0.100  0.688
model[T.no_shap]    -0.448    0.145 -3.098 0.002 -0.731 -0.165
model[T.shap]        0.190    0.135  1.409 0.159 -0.074  0.453
Group Var            0.291    0.122                           



### Housing

In [340]:
md = smf.mixedlm("q1_is_correct ~ model", random_effects_df_housing, groups=random_effects_df_housing['prolific_id'])
mdf = md.fit()
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: q1_is_correct
No. Observations: 194     Method:             REML         
No. Groups:       97      Scale:              0.1338       
Min. group size:  2       Log-Likelihood:     -107.3509    
Max. group size:  2       Converged:          Yes          
Mean group size:  2.0                                      
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.826    0.063 13.092 0.000  0.702  0.950
model[T.lr]      -0.257    0.083 -3.091 0.002 -0.419 -0.094
model[T.no_shap] -0.316    0.082 -3.846 0.000 -0.477 -0.155
model[T.shap]     0.137    0.081  1.684 0.092 -0.022  0.296
Group Var         0.037    0.058                           



In [341]:
md = smf.mixedlm("q2_is_correct ~ model", random_effects_df_housing, groups=random_effects_df_housing['prolific_id'])
mdf = md.fit()
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: q2_is_correct
No. Observations: 194     Method:             REML         
No. Groups:       97      Scale:              0.1692       
Min. group size:  2       Log-Likelihood:     -121.3178    
Max. group size:  2       Converged:          Yes          
Mean group size:  2.0                                      
-----------------------------------------------------------
                   Coef. Std.Err.   z   P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept          0.352    0.068 5.187 0.000  0.219  0.486
model[T.lr]        0.451    0.090 5.015 0.000  0.275  0.628
model[T.no_shap]   0.274    0.091 3.008 0.003  0.095  0.452
model[T.shap]      0.452    0.089 5.085 0.000  0.278  0.626
Group Var          0.026    0.055                          



In [342]:
md = smf.mixedlm("understood_system ~ model", random_effects_df_housing, groups=random_effects_df_housing['prolific_id'])
mdf = md.fit()
print(mdf.summary())

             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: understood_system
No. Observations: 194     Method:             REML             
No. Groups:       97      Scale:              0.3796           
Min. group size:  2       Log-Likelihood:     -218.8738        
Max. group size:  2       Converged:          Yes              
Mean group size:  2.0                                          
---------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------
Intercept             3.446    0.114 30.281 0.000  3.223  3.669
model[T.lr]          -0.270    0.148 -1.826 0.068 -0.560  0.020
model[T.no_shap]     -0.237    0.142 -1.672 0.094 -0.515  0.041
model[T.shap]         0.346    0.142  2.441 0.015  0.068  0.625
Group Var             0.194    0.129                           



In [343]:
md = smf.mixedlm("system_confusing ~ model", random_effects_df_housing, groups=random_effects_df_housing['prolific_id'])
mdf = md.fit()
print(mdf.summary())

            Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: system_confusing
No. Observations: 194     Method:             REML            
No. Groups:       97      Scale:              0.5911          
Min. group size:  2       Log-Likelihood:     -255.9199       
Max. group size:  2       Converged:          Yes             
Mean group size:  2.0                                         
--------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            3.516    0.138 25.548 0.000  3.246  3.785
model[T.lr]          0.157    0.179  0.876 0.381 -0.194  0.509
model[T.no_shap]     0.025    0.175  0.144 0.885 -0.317  0.368
model[T.shap]       -0.638    0.174 -3.664 0.000 -0.979 -0.297
Group Var            0.243    0.143                           



# Analysis (old)

In [37]:
def generate_stats(data):
    data['q1_is_correct_int'] = data['q1_is_correct_int'].apply(lambda x: 1 if x == True else 0)
    data['q1_is_correct_bb'] = data['q1_is_correct_bb'].apply(lambda x: 1 if x == True else 0)
    data['q2_is_correct_int'] = data['q2_is_correct_int'].apply(lambda x: 1 if x == True else 0)
    data['q2_is_correct_bb'] = data['q2_is_correct_bb'].apply(lambda x: 1 if x == True else 0)

    print("Profile Task 1:")
    print("  q1_is_correct_int_mean ", data.q1_is_correct_int.mean())
    print("  q1_is_correct_bb_mean ", data.q1_is_correct_bb.mean())
    print(pt.ttest(data.q1_is_correct_int, data.q1_is_correct_bb, paired=True))
    print("\n--------------------\n")
    print("Profile Task 2:")
    print("  q2_is_correct_int_mean ", data.q2_is_correct_int.mean())
    print("  q2_is_correct_bb_mean ", data.q2_is_correct_bb.mean())
    print(pt.ttest(data.q2_is_correct_int, data.q2_is_correct_bb, paired=True))
    print("\n--------------------\n")
    print("Understood System Construct:")
    print("  undersood_system_int_mean", data.understood_system_int.mean())
    print("  undersood_system_bb_mean ", data.understood_system_bb.mean())
    print(pt.ttest(data.understood_system_int, data.understood_system_bb, paired=True))
    print("\n--------------------\n")
    print("System Confusing Construct:")
    print("  system_confusing_int_mean", data.system_confusing_int.mean())
    print("  system_confusing_bb_mean ", data.system_confusing_bb.mean())
    print(pt.ttest(data.system_confusing_int, data.system_confusing_bb, paired=True))

In [38]:
'''
print("--------------------")

data_no_shap = data[data['bb_model'] == 'no_shap']
data_shap = data[data['bb_model'] == 'shap']

print("q1_is_correct_bb_no_shap_mean ", data_no_shap.q1_is_correct_bb.mean())
print("q1_is_correct_bb_shap_mean ", data_shap.q1_is_correct_bb.mean())
print(pt.ttest(data_no_shap.q1_is_correct_bb, data_shap.q1_is_correct_bb, paired=True))
print("--------------------")
print("q2_is_correct_bb_no_shap_mean ", data_no_shap.q2_is_correct_bb.mean())
print("q2_is_correct_bb_shap_mean ", data_shap.q2_is_correct_bb.mean())
print(pt.ttest(data_no_shap.q2_is_correct_bb, data_shap.q2_is_correct_bb, paired=True))
print("--------------------")
print("system_understanding_bb_no_shap_mean", data_no_shap.system_understanding_bb.mean())
print("system_understanding_bb_shap_mean ", data_shap.system_understanding_bb.mean())
print(pt.ttest(data_no_shap.system_understanding_bb, data_shap.system_understanding_bb, paired=True))
'''

'\nprint("--------------------")\n\ndata_no_shap = data[data[\'bb_model\'] == \'no_shap\']\ndata_shap = data[data[\'bb_model\'] == \'shap\']\n\nprint("q1_is_correct_bb_no_shap_mean ", data_no_shap.q1_is_correct_bb.mean())\nprint("q1_is_correct_bb_shap_mean ", data_shap.q1_is_correct_bb.mean())\nprint(pt.ttest(data_no_shap.q1_is_correct_bb, data_shap.q1_is_correct_bb, paired=True))\nprint("--------------------")\nprint("q2_is_correct_bb_no_shap_mean ", data_no_shap.q2_is_correct_bb.mean())\nprint("q2_is_correct_bb_shap_mean ", data_shap.q2_is_correct_bb.mean())\nprint(pt.ttest(data_no_shap.q2_is_correct_bb, data_shap.q2_is_correct_bb, paired=True))\nprint("--------------------")\nprint("system_understanding_bb_no_shap_mean", data_no_shap.system_understanding_bb.mean())\nprint("system_understanding_bb_shap_mean ", data_shap.system_understanding_bb.mean())\nprint(pt.ttest(data_no_shap.system_understanding_bb, data_shap.system_understanding_bb, paired=True))\n'

In [58]:
def generate_system_level_stats(data):
    print("Decision Tree")
    print("Profile Task 1 (q1_is_correct_int_mean) ", data[data['int_model']=='dt'].q1_is_correct_int.mean())
    print("Profile Task 2 (q2_is_correct_int_mean) ", data[data['int_model']=='dt'].q2_is_correct_int.mean())
    print("understood_system_int_mean ", data[data['int_model']=='dt'].understood_system_int.mean())
    print("system_confusing_int_mean ", data[data['int_model']=='dt'].system_confusing_int.mean())

    print("\n--------------------\n")

    print("Linear Model")
    print("Profile Task 1 (q1_is_correct_int_mean) ", data[data['int_model']=='lr'].q1_is_correct_int.mean())
    print("Profile Task 2 (q2_is_correct_int_mean) ", data[data['int_model']=='lr'].q2_is_correct_int.mean())
    print("understood_system_int_mean ", data[data['int_model']=='lr'].understood_system_int.mean())
    print("system_confusing_int_mean ", data[data['int_model']=='lr'].system_confusing_int.mean())


    print("\n--------------------\n")

    print("BB (no SHAP) Model")
    print("Profile Task 1 (q1_is_correct_bb_mean) ", data[data['bb_model']=='no_shap'].q1_is_correct_bb.mean())
    print("Profile Task 2 (q2_is_correct_bb_mean) ", data[data['bb_model']=='no_shap'].q2_is_correct_bb.mean())
    print("understood_system_bb_mean ", data[data['bb_model']=='no_shap'].understood_system_bb.mean())
    print("system_confusing_bb_mean ", data[data['bb_model']=='no_shap'].system_confusing_bb.mean())

    print("\n--------------------\n")

    print("BB (w/ SHAP) Model")
    print("Profile Task 1 (q1_is_correct_bb_mean) ", data[data['bb_model']=='shap'].q1_is_correct_bb.mean())
    print("Profile Task 2 (q2_is_correct_bb_mean) ", data[data['bb_model']=='shap'].q2_is_correct_bb.mean())
    print("understood_system_bb_mean ", data[data['bb_model']=='shap'].understood_system_bb.mean())
    print("system_confusing_bb_mean ", data[data['bb_model']=='shap'].system_confusing_bb.mean())
    
    print("\n--------------------\n")
    
    print("Profile Task 1:")
    temp = pd.DataFrame()
    temp['dt'] = data[data['int_model']=='dt'].q1_is_correct_int.reset_index(drop=True)
    temp['lr'] = data[data['int_model']=='lr'].q1_is_correct_int.reset_index(drop=True)
    temp['shap'] = data[data['bb_model']=='shap'].q1_is_correct_bb.reset_index(drop=True)
    temp['no_shap'] = data[data['bb_model']=='no_shap'].q1_is_correct_bb.reset_index(drop=True)
    print(pt.rm_anova(data=temp))
    
    print("Profile Task 2:")   
    temp = pd.DataFrame()
    temp['dt'] = data[data['int_model']=='dt'].q2_is_correct_int.reset_index(drop=True)
    temp['lr'] = data[data['int_model']=='lr'].q2_is_correct_int.reset_index(drop=True)
    temp['shap'] = data[data['bb_model']=='shap'].q2_is_correct_bb.reset_index(drop=True)
    temp['no_shap'] = data[data['bb_model']=='no_shap'].q2_is_correct_bb.reset_index(drop=True)
    print(pt.rm_anova(data=temp))
    
    print("Understood System Construct:")    
    temp = pd.DataFrame()
    temp['dt'] = data[data['int_model']=='dt'].understood_system_int.reset_index(drop=True)
    temp['lr'] = data[data['int_model']=='lr'].understood_system_int.reset_index(drop=True)
    temp['shap'] = data[data['bb_model']=='shap'].understood_system_bb.reset_index(drop=True)
    temp['no_shap'] = data[data['bb_model']=='no_shap'].understood_system_bb.reset_index(drop=True)
    print(pt.rm_anova(data=temp))
    
    print("System Confusing Construct:")    
    temp = pd.DataFrame()
    temp['dt'] = data[data['int_model']=='dt'].system_confusing_int.reset_index(drop=True)
    temp['lr'] = data[data['int_model']=='lr'].system_confusing_int.reset_index(drop=True)
    temp['shap'] = data[data['bb_model']=='shap'].system_confusing_bb.reset_index(drop=True)
    temp['no_shap'] = data[data['bb_model']=='no_shap'].system_confusing_bb.reset_index(drop=True)
    print(pt.rm_anova(data=temp))


## Whole sample (all datasets)

In [59]:
generate_stats(data)

Profile Task 1:
  q1_is_correct_int_mean  0.7722007722007722
  q1_is_correct_bb_mean  0.8262548262548263
               T  dof alternative     p-val          CI95%   cohen-d   BF10  \
T-test -1.729897  258   two-sided  0.084845  [-0.12, 0.01]  0.134987  0.303   

          power  
T-test  0.58098  

--------------------

Profile Task 2:
  q2_is_correct_int_mean  0.7644787644787645
  q2_is_correct_bb_mean  0.8416988416988417
               T  dof alternative     p-val           CI95%   cohen-d   BF10  \
T-test -2.448636  258   two-sided  0.015006  [-0.14, -0.02]  0.194729  1.305   

           power  
T-test  0.877425  

--------------------

Understood System Construct:
  undersood_system_int_mean 3.511627906976744
  undersood_system_bb_mean  3.668725868725868
               T  dof alternative     p-val           CI95%   cohen-d   BF10  \
T-test -2.877319  257   two-sided  0.004348  [-0.27, -0.05]  0.207239  3.922   

           power  
T-test  0.912503  

--------------------

System 

## Datasets by domain

#### Education

In [60]:
education_data = data[data['dataset_name'] == 'education']
generate_stats(education_data)

Profile Task 1:
  q1_is_correct_int_mean  0.8260869565217391
  q1_is_correct_bb_mean  0.8695652173913043
               T  dof alternative     p-val          CI95%   cohen-d   BF10  \
T-test -1.220394  160   two-sided  0.224111  [-0.11, 0.03]  0.120891  0.182   

           power  
T-test  0.331941  

--------------------

Profile Task 2:
  q2_is_correct_int_mean  0.8571428571428571
  q2_is_correct_bb_mean  0.9192546583850931
               T  dof alternative     p-val         CI95%   cohen-d   BF10  \
T-test -1.905194  160   two-sided  0.058549  [-0.13, 0.0]  0.197452  0.513   

           power  
T-test  0.702074  

--------------------

Understood System Construct:
  undersood_system_int_mean 3.6434782608695646
  undersood_system_bb_mean  3.7652173913043465
               T  dof alternative     p-val          CI95%   cohen-d   BF10  \
T-test -1.725313  160   two-sided  0.086402  [-0.26, 0.02]  0.160991  0.374   

           power  
T-test  0.528142  

--------------------

System Co

In [61]:
generate_system_level_stats(education_data)

Decision Tree
Profile Task 1 (q1_is_correct_int_mean)  0.8690476190476191
Profile Task 2 (q2_is_correct_int_mean)  0.7857142857142857
understood_system_int_mean  3.797619047619046
system_confusing_int_mean  3.0200803212851417

--------------------

Linear Model
Profile Task 1 (q1_is_correct_int_mean)  0.7792207792207793
Profile Task 2 (q2_is_correct_int_mean)  0.935064935064935
understood_system_int_mean  3.4753246753246763
system_confusing_int_mean  3.4761904761904763

--------------------

BB (no SHAP) Model
Profile Task 1 (q1_is_correct_bb_mean)  0.9571428571428572
Profile Task 2 (q2_is_correct_bb_mean)  0.8714285714285714
understood_system_bb_mean  3.991428571428571
system_confusing_bb_mean  2.614285714285714

--------------------

BB (w/ SHAP) Model
Profile Task 1 (q1_is_correct_bb_mean)  0.8021978021978022
Profile Task 2 (q2_is_correct_bb_mean)  0.9560439560439561
understood_system_bb_mean  3.59120879120879
system_confusing_bb_mean  3.230769230769231

--------------------

Profil

In [62]:
#temp = pd.DataFrame()
#temp['dt'] = education_data[education_data['int_model']=='dt'].q1_is_correct_int.reset_index(drop=True)
#temp['lr'] = education_data[education_data['int_model']=='lr'].q1_is_correct_int.reset_index(drop=True)
#temp['shap'] = education_data[education_data['bb_model']=='shap'].q1_is_correct_bb.reset_index(drop=True)
#temp['no_shap'] = education_data[education_data['bb_model']=='no_shap'].q1_is_correct_bb.reset_index(drop=True)

#### Housing

In [63]:
housing_data = data[data['dataset_name'] == 'housing']
generate_stats(housing_data)

Profile Task 1:
  q1_is_correct_int_mean  0.6836734693877551
  q1_is_correct_bb_mean  0.7551020408163265
             T  dof alternative     p-val          CI95%   cohen-d  BF10  \
T-test -1.2216   97   two-sided  0.224819  [-0.19, 0.04]  0.158667  0.23   

           power  
T-test  0.343037  

--------------------

Profile Task 2:
  q2_is_correct_int_mean  0.6122448979591837
  q2_is_correct_bb_mean  0.7142857142857143
               T  dof alternative   p-val          CI95%   cohen-d   BF10  \
T-test -1.593508   97   two-sided  0.1143  [-0.23, 0.03]  0.216075  0.379   

          power  
T-test  0.56274  

--------------------

Understood System Construct:
  undersood_system_int_mean 3.292783505154638
  undersood_system_bb_mean  3.510204081632655
               T  dof alternative    p-val           CI95%   cohen-d   BF10  \
T-test -2.449701   96   two-sided  0.01611  [-0.41, -0.04]  0.289339  1.909   

           power  
T-test  0.805403  

--------------------

System Confusing Cons

In [64]:
generate_system_level_stats(housing_data)

Decision Tree
Profile Task 1 (q1_is_correct_int_mean)  0.8333333333333334
Profile Task 2 (q2_is_correct_int_mean)  0.35714285714285715
understood_system_int_mean  3.4904761904761905
system_confusing_int_mean  3.5079365079365084

--------------------

Linear Model
Profile Task 1 (q1_is_correct_int_mean)  0.5714285714285714
Profile Task 2 (q2_is_correct_int_mean)  0.8035714285714286
understood_system_int_mean  3.1418181818181816
system_confusing_int_mean  3.678571428571428

--------------------

BB (no SHAP) Model
Profile Task 1 (q1_is_correct_bb_mean)  0.5111111111111111
Profile Task 2 (q2_is_correct_bb_mean)  0.6222222222222222
understood_system_bb_mean  3.2088888888888882
system_confusing_bb_mean  3.533333333333332

--------------------

BB (w/ SHAP) Model
Profile Task 1 (q1_is_correct_bb_mean)  0.9622641509433962
Profile Task 2 (q2_is_correct_bb_mean)  0.7924528301886793
understood_system_bb_mean  3.7660377358490575
system_confusing_bb_mean  2.893081761006289

--------------------

P

#### Healthcare

In [ ]:
healthcare_data = data[data['dataset_name'] == 'healthcare']
generate_stats(healthcare_data)

## Comparison across datasets

In [ ]:
print("education_q1_is_correct_int_mean ", education_data.q1_is_correct_int.mean())
print("housing_q1_is_correct_int_mean ", housing_data.q1_is_correct_int.mean())
print("healtcare_q1_is_correct_int_mean ", healthcare_data.q1_is_correct_int.mean())
print(f_oneway(education_data.q1_is_correct_int, housing_data.q1_is_correct_int,healthcare_data.q1_is_correct_int))
print("--------------------")
print("education_q2_is_correct_int_mean ", education_data.q2_is_correct_int.mean())
print("housing_q2_is_correct_int_mean ", housing_data.q2_is_correct_int.mean())
print("healtcare_q2_is_correct_int_mean ", healthcare_data.q2_is_correct_int.mean())
print(f_oneway(education_data.q2_is_correct_int, housing_data.q2_is_correct_int,healthcare_data.q2_is_correct_int))
print("--------------------")
print("education_system_understanding_int_mean", education_data.system_understanding_int.mean())
print("housing_system_understanding_int_mean", housing_data.system_understanding_int.mean())
print("healthcare_system_understanding_int_mean", healthcare_data.system_understanding_int.mean())
print(f_oneway(education_data.system_understanding_int,housing_data.system_understanding_int,healthcare_data.system_understanding_int))

In [ ]:
print("education_q1_is_correct_bb_mean ", education_data.q1_is_correct_bb.mean())
print("housing_q1_is_correct_bb_mean ", housing_data.q1_is_correct_bb.mean())
print("healtcare_q1_is_correct_bb_mean ", healthcare_data.q1_is_correct_bb.mean())
print(f_oneway(education_data.q1_is_correct_bb, housing_data.q1_is_correct_bb,healthcare_data.q1_is_correct_bb))
print("--------------------")
print("education_q2_is_correct_bb_mean ", education_data.q2_is_correct_bb.mean())
print("housing_q2_is_correct_bb_mean ", housing_data.q2_is_correct_bb.mean())
print("healtcare_q2_is_correct_bb_mean ", healthcare_data.q2_is_correct_bb.mean())
print(f_oneway(education_data.q2_is_correct_bb, housing_data.q2_is_correct_bb,healthcare_data.q2_is_correct_bb))
print("--------------------")
print("education_system_understanding_bb_mean", education_data.system_understanding_bb.mean())
print("housing_system_understanding_bb_mean", housing_data.system_understanding_bb.mean())
print("healthcare_system_understanding_bb_mean", healthcare_data.system_understanding_bb.mean())
print(f_oneway(education_data.system_understanding_bb,housing_data.system_understanding_bb,healthcare_data.system_understanding_bb))

## Subgroups

In [ ]:
knowledge_mapper = {
    "Very knowledgeable": 5,
    "Knowledgeable": 4,
    "Moderately knowledgeable": 3,
    "Slightly knowledgeable": 2,
    "Not knowledgeable at all": 1
}

data['Q9.2']
#print(data['Q9.2'].value_counts())
data['background_knowledge'] = data['Q9.2'].map(knowledge_mapper)
data['background_knowledge'].value_counts()

generate_stats(data[data['background_knowledge']>=3])

data['12_min_cutoff_flag'] = data['duration'].apply(lambda x: x >= 600)
print(data['12_min_cutoff_flag'].value_counts())
generate_stats(data[data['12_min_cutoff_flag'] == 1])

## Save datasets

In [197]:
education_data.head()

,StartDate,EndDate,Status,IPAddress,Progress,duration,Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q2.1_Id,Q2.1_Name,Q2.1_Size,Q2.1_Type,prolific_id,dt_q1,dt_q2,understood_system_did_well_dt,system_confusing_complicated_dt,system_confusing_confusing_dt,understood_system_easy_to_understand_dt,system_confusing_hard_to_remember_dt,understood_system_predictable_dt,understood_system_how_it_works_dt,understood_system_inputs_and_outputs_dt,lr_q1,lr_q2,understood_system_did_well_lr,system_confusing_complicated_lr,system_confusing_confusing_lr,understood_system_easy_to_understand_lr,system_confusing_hard_to_remember_lr,understood_system_predictable_lr,understood_system_how_it_works_lr,understood_system_inputs_and_outputs_lr,bb_no_shap_q1,bb_no_shap_q2,understood_system_did_well_bb_no_shap,system_confusing_complicated_bb_no_shap,system_confusing_confusing_bb_no_shap,understood_system_easy_to_understand_bb_no_shap,system_confusing_hard_to_remember_bb_no_shap,understood_system_predictable_bb_no_shap,understood_system_how_it_works_bb_no_shap,understood_system_inputs_and_outputs_bb_no_shap,bb_shap_q1,bb_shap_q2,understood_system_did_well_bb_shap,system_confusing_complicated_bb_shap,system_confusing_confusing_bb_shap,understood_system_easy_to_understand_bb_shap,system_confusing_hard_to_remember_bb_shap,understood_system_predictable_bb_shap,understood_system_how_it_works_bb_shap,understood_system_inputs_and_outputs_bb_shap,Q9.1,Q9.2,Q9.3_1,Q9.3_2,Q9.3_3,Q9.3_4,Q9.4,PROLIFIC_PID,dataset_name,bb_profile_1,bb_profile_2,int_profile_1,int_profile_2,first_model,bb_model,int_model,taken_in_appropiate_time_window,int_q1_correct_answer,int_q2_correct_answer,bb_q1_correct_answer,bb_q2_correct_answer,q1_is_correct_int,q1_is_correct_bb,q2_is_correct_int,q2_is_correct_bb,understood_system_bb,understood_system_int,system_confusing_bb,system_confusing_int
0,2021-08-23 22:54:23,2021-08-23 23:00:14,IP Address,104.35.20.232,100,351,True,2021-08-23 23:00:15,R_yqnE1JHcvGdFz7H,NaN,NaN,NaN,NaN,33.92999267578125,-117.94770050048828125,anonymous,EN,1,F_z6iKgOtuJxyuhMZ,signature.png,14093,image/png,6102d3c05c31cb5c38158b34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,5.0,5.0,4.0,2.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,First year grade,5.0,5.0,5.0,2.0,4.0,4.0,4.0,4.0,8,Not knowledgeable at all,No,No,Yes,No,No,6102d3c05c31cb5c38158b34,education,2,4,1,4,int,shap,lr,True,Fail,First year grade,Pass,First year grade,1,1,1,1,3.8,3.8,4.666667,3.666667
1,2021-08-23 23:24:31,2021-08-23 23:31:44,IP Address,47.147.135.146,100,433,True,2021-08-23 23:31:45,R_2sErBZc7mlGv79v,NaN,NaN,NaN,NaN,33.7902984619140625,-118.12149810791015625,anonymous,EN,1,F_1P8lvw788zdBWFh,signature.png,14986,image/png,610b88a14eed14f2b8b17b86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,Number of absences,2.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,Fail,First year grade,4.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Not knowledgeable at all,No,No,No,No,No,610b88a14eed14f2b8b17b86,education,1,3,4,1,bb,no_shap,lr,True,Pass,First year grade,Fail,First year grade,1,1,0,1,4.0,2.4,2.666667,4.000000
2,2021-08-23 23:55:52,2021-08-24 00:01:24,IP Address,68.172.164.31,100,332,True,2021-08-24 00:01:26,R_BEeWM93HGxSRK8N,NaN,NaN,NaN,NaN,43.2122955322265625,-77.69950103759765625,anonymous,EN,0.89999997615814208984375,F_1cTaUoQcmSH3xbZ,signature.png,12886,image/png,610c24152939c927ad421fa9,Pass,First year grade,4.0,2.0,1.0,5.0,4.0,3.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,First year grade,4.0,1.0,1.0,5.0,2.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Slightly knowledgeable,No,No,Yes,No,No,610c24152939c927ad421fa9,education,3,2,4,3,int,no_shap,dt,True,Pass,First year grade,Fail,First year grade,1,1,1,1,4.6,4.4,1.333333,2.333333
3,2021-08-24 00:12:19,2021-08-24 00:21:13,IP Address,24.165.25.108,100,

In [65]:
education_data.to_csv('analysis/education_analysis_data.csv')

In [66]:
data.to_csv('analysis/analysis_data.csv')